In [1]:
import argparse
import os
import warnings

import torch
import transformers
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    StoppingCriteria,
    StoppingCriteriaList,
)

warnings.filterwarnings("ignore")
torch.cuda.empty_cache()

2024-06-08 16:33:30.902388: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 16:33:30.930992: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 16:33:31.422146: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model_id = "KRAFTON/KORani-v3-13B"

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        offload_folder="./offload",
        llm_int8_enable_fp32_cpu_offload=True,
        low_cpu_mem_usage=True,
    )


model = load_checkpoint_and_dispatch(
    model,
    '/home/anderson/.cache/huggingface/hub/models--KRAFTON--KORani-v3-13B/',
    device_map="auto",
    offload_folder="./offload",
    # max_memory={0: "1600MB"},  # 최대 메모리 설정 적용
)


# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="balanced_low_0",
#     torch_dtype=torch.float16,  # Half Precision
#     # low_cpu_mem_usage=True,
#     offload_folder="./offload",
#     llm_int8_enable_fp32_cpu_offload=True,
# )
# Gradient Checkpointing 활성화
# model.gradient_checkpointing_enable()

model.eval()

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at yanolja/EEVE-Korean-10.8B-v1.0 and are newly initialized: ['model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.46.self_attn.rotary_emb.inv_freq', 'model.layers.37.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.44.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.34.self_attn.rotary_emb.inv_freq', 'model.layers.39.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.40.self_attn.rotary_emb.inv_freq', 'model.layers.41.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.43.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.ro

  0%|          | 0/96 [00:00<?, ?w/s]

  0%|          | 0/100 [00:00<?, ?w/s]

  0%|          | 0/106 [00:00<?, ?w/s]

  0%|          | 0/100 [00:00<?, ?w/s]

  0%|          | 0/33 [00:00<?, ?w/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [20]:
def calculate_balanced_device_map(model, prob: float = 0.5, output="./device_map.json"):
    import json

    layer_names = model.state_dict().keys()
    half_index = int(len(layer_names) * prob)

    device_map = {}
    for idx, layer_name in enumerate(layer_names):
        if idx < half_index:
            device_map[layer_name] = "cuda:0"
        else:
            device_map[layer_name] = "cpu"
    with open(output, "wt") as f:
        json.dump(device_map, f)


calculate_balanced_device_map(model)

In [4]:
from pprint import pprint

prompt = """
우리는 아래와 같은 정보를 갖고 있습니다.
---------------------
메시는 주장으로서 2014년 FIFA 월드컵과 2022년 FIFA 월드컵에서 아르헨티나 축구 국가대표팀을 이끌며 월드컵 결승에 진출하였으며, 2014년에는 아쉽게 독일에 패배했다. 2014년 대회에서 메시는 4경기 연속 맨 오브 더 매치에 선정되는 등 뛰어난 활약을 보이며 아르헨티나 축구 국가대표팀을 준우승으로 이끌었고 7경기 4골 1도움을 기록하며 2014 FIFA 월드컵 득점 3위에 오르고 월드컵 최우수 선수상인 골든볼을 수상했다. 그러나 2022년 대회에서는 디펜딩 챔피언 프랑스를 꺾고 우승을 차지하였으며, 역사상 최초로 골든볼을 2회 수상한 선수가 되었다.
리오넬 메시의 기록으로는 UEFA 챔피언스리그 4년 연속 득점왕, UEFA 챔피언스리그 통산 득점왕 5회(최다 통산 득점왕은 호날두 6회-5회 연속), UEFA 챔피언스리그 4회 우승, 단일 시즌 최다골 (60경기 73골), 단일 시즌 최다 공격포인트 (60경기 73골 29도움, 102공격포인트), 한해 최다골 (69경기 91골), 스페인 프리메라리가 통산 역대 득점 1위, 스페인 프리메라리가 통산 역대 도움 1위, FIFA 발롱도르 4년 연속으로 수상, UEFA 올해의 선수상 2회 수상, 월드컵 4경기 연속 맨 오브 더 매치, 월드컵 1회 우승과 1회 준우승, 월드컵 골든볼 수상 등이 있다.
2014-15 시즌에는 스페인 프리메라리가, 스페인 국왕컵, UEFA 챔피언스리그 3개의 중요 대회에서 우승을 차지하며 2008-09 시즌에 이어 또다시 트레블을 달성하였으며, 트레블 우승과 더불어 UEFA 챔피언스리그 득점왕과 UEFA 챔피언스리그 도움왕, 스페인 프리메라리가 도움왕을 차지했다. 이로써 메시는 2015년 한 해 동안 최고의 활약을 펼치며 소속팀 FC 바르셀로나를 이끌었고, FC 바르셀로나는 통산 5번째 UEFA 챔피언스리그 우승과 트레블 2회라는 기록을 달성했다. 2014-15 시즌에 57경기 58골을 기록한 메시는 프리메라리가 최우수선수상, UEFA 올해의 선수상 등 각종 상을 싹쓸이하였다. 그리고 이러한 활약에 힘입어 2015년 FIFA 발롱도르를 수상했으며, 이는 메시의 5번째 FIFA 발롱도르이다.
---------------------
### 주어진 정보에 따라, 질문에 답해주세요.: '메시가 트레블을 달성한 시즌은 언제야?'
### Assistant:
"""

batch = tokenizer(prompt, return_tensors="pt")
prompt_size = len(batch["input_ids"][0])
print("prompt_size:", prompt_size)
batch = {k: v.to() for k, v in batch.items()}
pprint(batch)

generation_config = GenerationConfig(
    temperature=0.1,
    max_new_tokens=32,
    exponential_decay_length_penalty=(256, 1.03),
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.05,
    do_sample=True,
    top_p=0.7,
    min_length=5,
    use_cache=True,
    return_dict_in_generate=True,
)

torch.cuda.empty_cache()
with torch.no_grad():
    generated = model.generate(**batch, generation_config=generation_config)
    response = tokenizer.decode(
        generated["sequences"][0][prompt_size:], skip_special_tokens=True
    )

prompt_size: 756
{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

ValueError: The following `model_kwargs` are not used by the model: ['token_type_ids'] (note: typos in the generate arguments will also show up in this list)

In [4]:
batch

{'input_ids': tensor([[    2, 29871,    13,  ..., 22137, 29901,    13]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [2]:
import torch
import transformers

model_id = "KRAFTON/KORani-v3-13B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

PROMPT = """당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner."""
instruction = "서울과학기술대학교 MLP연구실에 대해 소개해줘"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"},
]

prompt = pipeline.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt) :])

ValueError: Could not load model KRAFTON/KORani-v3-13B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>).

In [7]:
import torch
import torch_tensorrt


# 간단한 모델 정의 (스크립팅 가능)
class SimpleModel(torch.nn.Module):
    def forward(self, x):
        return x + 1


model = SimpleModel().cuda().eval()

# JIT 스크립트 모듈로 변환
scripted_model = torch.jit.script(model)

# TensorRT 변환
input_tensor = torch.randn((1, 3, 224, 224)).cuda()
trt_model = torch_tensorrt.ts.compile(
    scripted_model, inputs=[torch_tensorrt.Input(input_tensor.shape)]
)

# 변환된 모델로 추론
with torch.no_grad():
    output = trt_model(input_tensor)

print("TensorRT 변환 및 추론 성공:", output.shape)

TensorRT 변환 및 추론 성공: torch.Size([1, 3, 224, 224])
